1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd
import numpy as np
file_name = "./data/data_000637.txt"
data = pd.read_csv(file_name)
print('Max DataSize:',data.shape[0])
N = np.random.randint(10000, data.shape[0])
df = data[:1310720] # for max N , use df = data[:N] for a randomized dataframe
df

Max DataSize: 1310720


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14
1310716,1,1,4,3869211171,763,11
1310717,1,0,64,3869211171,764,0
1310718,1,0,139,3869211171,769,0


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [3]:
x = df['BX_COUNTER'].max()
x

3563

3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [4]:
tdc_count_ns = 25/30
bx_unit_ns = 25 
dfa = df.copy()

orbit_time = (dfa['ORBIT_CNT'] * bx_unit_ns * x) - (dfa.loc[0,'ORBIT_CNT'] * bx_unit_ns * x)
bx_time = (dfa['BX_COUNTER'] * bx_unit_ns) - (dfa['BX_COUNTER'] * bx_unit_ns)[0]
tdc = (dfa['TDC_MEAS'] * tdc_count_ns) - (dfa['TDC_MEAS'] * tdc_count_ns)[0]

dfa['TIME'] = (orbit_time.cumsum() + bx_time.cumsum() + tdc.cumsum())
dfa['TIME_STAMP'] = pd.to_timedelta(dfa['TIME'], unit = 'ns') # Time difference from the beginning of acquisition

# To time series

time_series = pd.Series(dfa['TIME_STAMP'])
dfa

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME,TIME_STAMP
0,1,0,123,3869200167,2374,26,0.000000e+00,0 days 00:00:00
1,1,0,124,3869200167,2374,27,8.333333e-01,0 days 00:00:00
2,1,0,63,3869200167,2553,28,4.477500e+03,0 days 00:00:00.000004477
3,1,0,64,3869200167,2558,19,9.071667e+03,0 days 00:00:00.000009071
4,1,0,64,3869200167,2760,25,1.872083e+04,0 days 00:00:00.000018720
...,...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,7.033920e+14,8 days 03:23:12.043346506
1310716,1,1,4,3869211171,763,11,7.033930e+14,8 days 03:23:13.023487519
1310717,1,0,64,3869211171,764,0,7.033940e+14,8 days 03:23:14.003628547
1310718,1,0,139,3869211171,769,0,7.033950e+14,8 days 03:23:14.983769700


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [5]:
#find the duration of the data
print('\nTime Series Data:')
print(time_series)

print('\nTotal Time taken in data acquisition:')
print(dfa['TIME_STAMP'][len(dfa)-1] - dfa['TIME_STAMP'][0])


Time Series Data:
0                   0 days 00:00:00
1                   0 days 00:00:00
2         0 days 00:00:00.000004477
3         0 days 00:00:00.000009071
4         0 days 00:00:00.000018720
                     ...           
1310715   8 days 03:23:12.043346506
1310716   8 days 03:23:13.023487519
1310717   8 days 03:23:14.003628547
1310718   8 days 03:23:14.983769700
1310719   8 days 03:23:15.963910694
Name: TIME_STAMP, Length: 1310720, dtype: timedelta64[ns]

Total Time taken in data acquisition:
8 days 03:23:15.963910694


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [6]:
noisy_channels = df.groupby('TDC_CHANNEL')['ORBIT_CNT'].count().sort_values(ascending = False)
noisy_channels[0:3]

TDC_CHANNEL
139    108059
64      66020
63      64642
Name: ORBIT_CNT, dtype: int64

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [7]:
print('\nGrouping By orbits:')
print(df.groupby('ORBIT_CNT').count())

# Counting the orbits in the group
print('Number of non-empty orbits:',len(df.groupby('ORBIT_CNT').count()))


Grouping By orbits:
            HEAD  FPGA  TDC_CHANNEL  BX_COUNTER  TDC_MEAS
ORBIT_CNT                                                
3869200167    43    43           43          43        43
3869200168    85    85           85          85        85
3869200169   127   127          127         127       127
3869200170    98    98           98          98        98
3869200171   109   109          109         109       109
...          ...   ...          ...         ...       ...
3869211167   208   208          208         208       208
3869211168   109   109          109         109       109
3869211169   191   191          191         191       191
3869211170   137   137          137         137       137
3869211171    22    22           22          22        22

[11001 rows x 5 columns]
Number of non-empty orbits: 11001


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [8]:
print('Number of unique orbits with at least one measurement from TDC_CHANNEL=139:')
df[df['TDC_CHANNEL'] == 139].groupby('ORBIT_CNT').count().shape[0]

Number of unique orbits with at least one measurement from TDC_CHANNEL=139:


10976

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [9]:
fpga0 = df[df['FPGA'] == 0].groupby(['TDC_CHANNEL'])['TDC_CHANNEL'].count()
fpga1 = df[df['FPGA'] == 1].groupby(['TDC_CHANNEL'])['TDC_CHANNEL'].count()

print("\nFPGA0 : \n",fpga0)
print("\nFPGA0 type : \n",type(fpga0))

print("\nFPGA1 : \n",fpga1)
print("\nFPGA1 type : \n",type(fpga1))


FPGA0 : 
 TDC_CHANNEL
1       1215
2       1602
3       1493
4       2352
5       1432
       ...  
129        2
130       33
137       32
138       34
139    75617
Name: TDC_CHANNEL, Length: 124, dtype: int64

FPGA0 type : 
 <class 'pandas.core.series.Series'>

FPGA1 : 
 TDC_CHANNEL
1      28438
2      32669
3      21970
4      26403
5      15003
       ...  
129       35
130       38
137       36
138       36
139    32442
Name: TDC_CHANNEL, Length: 132, dtype: int64

FPGA1 type : 
 <class 'pandas.core.series.Series'>


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.